In [1]:
#@title ## Colab Mobile Keep Alive 
#@markdown Jalankan Agar Colab Mobile tetap aktif

%%html
<b>Press play on the music player to keep the tab alive before running the start cell (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title Install WebUI { display-mode: "form" }
#@markdown <b><font size="4">Jalankan ini untuk memulai instalasi.
import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

#@markdown Centang jika ingin menggunakan/menyimpan saved Images di Gdrive (Wajib di aktifkan jika ingin menggunakan fitur GDrive lain nya)
useGDrive = True #@param {type:"boolean"}
if useGDrive:
  from google.colab import drive
  drive.mount('/content/drive')

#@markdown <br/>
#@markdown <br/>
#@markdown <br/>
#@markdown <b><font size="4">(Optional) GDrive Configuration (Wajib Menyalakan Fungsi useGdrive sebelum fungsi dibawah ini)</b>
#@markdown <br/>
#@markdown <br/>

#@markdown Centang jika ingin menggunakan/menyimpan Checkpoint, Lora & Textual Inversion di Gdrive
saveModels = True #@param {type:"boolean"}

#@markdown Centang jika ingin menggunakan/menyimpan Controlnet & CLIP di Gdrive
#@markdown <br/>
#@markdown <b>(PERINGATAN FUNGSI INI MENGGUNAKAN 10GB STORAGE)
saveCNCLIP = False #@param {type:"boolean"}

#@markdown (EXPERIMENTAL) Centang jika ingin menggunakan/menyimpan Config di GDrive
saveConfig = False #@param {type:"boolean"}

if not useGDrive and (saveModels or saveCNCLIP or saveConfig):
  print("Peringatan: useGdrive tidak aktif tapi anda menyalakn fitur simpan di Gdrive. Harap periksa kembali.")
  import sys
  sys.exit('Proses Dihentikan')

print("Update & Install Aria2 & UnionFS")
!apt update -yqq
!apt install aria2 unionfs-fuse -yqq

print("Upgrade Python to 3.10.9")
!wget https://repo.anaconda.com/miniconda/Miniconda3-py310_23.1.0-1-Linux-x86_64.sh
!bash ./Miniconda3-py310_23.1.0-1-Linux-x86_64.sh -b -f -p /usr/local
!rm *.sh

print("Preparing libtcmalloc.so untuk optimalkan memory alocation")
!apt install libunwind8-dev -yqq
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/google-perftools_2.7-1ubuntu2_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/libgoogle-perftools-dev_2.7-1ubuntu2_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/libgoogle-perftools4_2.7-1ubuntu2_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/libtcmalloc-minimal4_2.7-1ubuntu2_amd64.deb
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

print("Install Latest PyTorch Family")
%pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
%pip install -q xformers==0.0.18 triton==2.0.0 gdown -U

print("Clone semua repository yang dibutuhkan.")
!git clone https://github.com/aikazu/stable-diffusion-webui /content/sdwebui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/sdwebui/extensions/sdwebui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/sdwebui/extensions/sdwebui-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser /content/sdwebui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/sdwebui/extensions/sd-webui-additional-networks
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/sdwebui/extensions/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor /content/sdwebui/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib /content/sdwebui/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex /content/sdwebui/extensions/posex
!git clone https://github.com/camenduru/sd-webui-tunnels /content/sdwebui/extensions/sd-webui-tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/sdwebui/extensions/batchlinks-webui
#!git clone https://github.com/d8ahazard/sd_dreambooth_extension /content/sdwebui/extensions/sd_dreambooth_extension
%cd /content/sdwebui

if not os.path.exists(f"/content/sdwebui/models/Lora"):
  os.mkdir(f"/content/sdwebui/models/Lora")
if not os.path.exists(f"/content/sdwebui/log/"):
  os.mkdir(f"/content/sdwebui/log/")
if not os.path.exists(f"/content/sdwebui/log/images"):
  os.mkdir(f"/content/sdwebui/log/images")

if saveModels:
  if not os.path.exists(f"/content/drive/MyDrive/SDdata"):
    os.mkdir(f"/content/drive/MyDrive/SDdata")
  if not os.path.exists(f"/content/drive/MyDrive/SDdata/Checkpoint"):
    os.mkdir(f"/content/drive/MyDrive/SDdata/Checkpoint")
  if not os.path.exists(f"/content/drive/MyDrive/SDdata/Lora"):
    os.mkdir(f"/content/drive/MyDrive/SDdata/Lora")
  if not os.path.exists(f"/content/drive/MyDrive/SDdata/Embeddings"):
    os.mkdir(f"/content/drive/MyDrive/SDdata/Embeddings")
  !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/Checkpoint=RW /content/sdwebui/models/Stable-diffusion
  !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/Lora=RW /content/sdwebui/extensions/sd-webui-additional-networks/models/lora
  !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/Lora=RW /content/sdwebui/models/Lora
  !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/Embeddings=RW /content/sdwebui/embeddings

print("Checking Checkpoint")
has_checkpoint_file = False
for filename in os.listdir(f"/content/sdwebui/models/Stable-diffusion"):
    if filename.endswith(".safetensors"):
        has_checkpoint_file = True
    if not has_checkpoint_file:
        print(f"Tidak ada Checkpoint terdeteksi. prepare download.")
        !gdown 1KNU_giorZYOflT3gdwAPtIPh6sNE2oNR -O /content/sdwebui/models/Stable-diffusion/Chilled_re_generic_v2.safetensors

if saveCNCLIP:
    if not os.path.exists(f"/content/drive/MyDrive/SDdata/Controlnet"):
        os.mkdir(f"/content/drive/MyDrive/SDdata/Controlnet")
    if not os.path.exists(f"/content/drive/MyDrive/SDdata/CLIP"):
        os.mkdir(f"/content/drive/MyDrive/SDdata/CLIP")
    if not os.path.exists(f"/content/sdwebui/models/CLIP"):
        os.mkdir(f"/content/sdwebui/models/CLIP")
    !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/Controlnet/=RW /content/sdwebui/extensions/sd-webui-controlnet/
    !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/CLIP=RW /content/sdwebui/models/CLIP

print("Checking ControlNet")
has_controlnet_file = False
for filename in os.listdir(f"/content/sdwebui/extensions/sd-webui-controlnet/models"):
    if filename.endswith(".safetensors"):
        has_controlnet_file = True
if not has_controlnet_file:
    print(f"Tidak ada Controlnet terdeteksi. prepare download.")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/sdwebui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/sdwebui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/sdwebui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth

print("Checking CLIP")
if not os.path.exists(f"/content/sdwebui/models/CLIP"):
    os.mkdir(f"/content/sdwebui/models/CLIP")
has_CLIP_file = False
for filename in os.listdir(f"/content/sdwebui/models/CLIP"):
    if filename.endswith(".pt"):
        has_CLIP_file = True
if not has_CLIP_file:
    print(f"Tidak ada CLIP terdeteksi. prepare download.")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt -d /content/sdwebui/models/CLIP -o ViT-L-14.pt

!sed -i -e 's/    start()/    #start()/g' /content/sdwebui/launch.py
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/sdwebui/launch.py
!python launch.py --skip-torch-cuda-test

print("Last touch!")
!git reset --hard
!git pull
!git -C /content/sdwebui/extensions/sdwebui-images-browser reset --hard
!git -C /content/sdwebui/extensions/sdwebui-images-browser pull
!git -C /content/sdwebui/extensions/sdwebui-huggingface reset --hard
!git -C /content/sdwebui/extensions/sdwebui-huggingface pull
!git -C /content/sdwebui/extensions/sd-civitai-browser reset --hard
!git -C /content/sdwebui/extensions/sd-civitai-browser pull
!git -C /content/sdwebui/extensions/sd-webui-additional-networks reset --hard
!git -C /content/sdwebui/extensions/sd-webui-additional-networks pull
!git -C /content/sdwebui/extensions/sd-webui-controlnet reset --hard
!git -C /content/sdwebui/extensions/sd-webui-controlnet pull
!git -C /content/sdwebui/extensions/openpose-editor reset --hard
!git -C /content/sdwebui/extensions/openpose-editor pull
!git -C /content/sdwebui/extensions/sd-webui-depth-lib reset --hard
!git -C /content/sdwebui/extensions/sd-webui-depth-lib pull
!git -C /content/sdwebui/extensions/posex reset --hard
!git -C /content/sdwebui/extensions/posex pull
!git -C /content/sdwebui/extensions/sd-webui-tunnels reset --hard
!git -C /content/sdwebui/extensions/sd-webui-tunnels pull
!git -C /content/sdwebui/extensions/batchlinks-webui reset --hard
!git -C /content/sdwebui/extensions/batchlinks-webui pull
#!git -C /content/sdwebui/extensions/sd_dreambooth_extension reset --hard
#!git -C /content/sdwebui/extensions/sd_dreambooth_extension pull

if useGDrive:
    if not os.path.exists(f"/content/drive/MyDrive/SDdata"):
        os.mkdir(f"/content/drive/MyDrive/SDdata")
    if not os.path.exists(f"/content/drive/MyDrive/SDdata/SavedImages"):
        os.mkdir(f"/content/drive/MyDrive/SDdata/SavedImages")
    if not os.path.exists(f"/content/drive/MyDrive/SDdata/SavedConfig"):
        os.mkdir(f"/content/drive/MyDrive/SDdata/SavedConfig")
    !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/SavedImages=RW /content/sdwebui/log/images
    !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/SavedConfig=RW /content/sdwebui/configs
    if os.listdir(f"/content/drive/MyDrive/SDdata/SavedConfig"):
      print("Config Ditemukan.")
    else:
      !git reset --hard
      !git pull

if saveConfig:
  if not os.path.exists(f"/content/drive/MyDrive/SDdata"):
    os.mkdir(f"/content/drive/MyDrive/SDdata")
  if not os.path.exists(f"/content/drive/MyDrive/SDdata/SavedConfig"):
    os.mkdir(f"/content/drive/MyDrive/SDdata/SavedConfig")
  !unionfs-fuse -o nonempty /content/drive/MyDrive/SDdata/SavedConfig=RW /content/sdwebui/configs
  if os.listdir(f"/content/drive/MyDrive/SDdata/SavedConfig"):
    print("Config Ditemukan.")
  else:
    !git reset --hard
    !git pull

!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/sdwebui/launch.py
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/sdwebui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/sdwebui/launch.py
!sed -i -e 's/accelerate==0.12.0/accelerate==0.18.0/g' /content/sdwebui/requirements_versions.txt
!sed -i -e 's/transformers==4.25.1/transformers==4.27.4/g' /content/sdwebui/requirements_versions.txt
!sed -i -e 's/fastapi==0.94.0/fastapi==0.95.0/g' /content/sdwebui/requirements_versions.txt

#!sed -i -e 's/transformers~=4.26.1/transformers==4.27.4/g' /content/sdwebui/extensions/sd_dreambooth_extension/requirements.txt
#!sed -i -e 's/fastapi~=0.94.1/fastapi==0.95.0/g' /content/sdwebui/extensions/sd_dreambooth_extension/requirements.txt

print("Done! silahkan jalankan Run WebUI")

Mounted at /content/drive
Update & Install Aria2 & UnionFS
30 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2 unionfs-fuse
0 upgraded, 4 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,524 kB of archives.
After this operation, 6,105 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 122352 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking ar

In [ ]:
#@title Run WebUI { display-mode: "form" }
#@markdown Jalankan ini untuk memulai WebUI setelah proses instalasi selesai. rekomendasi menggunakan link moe

!python launch.py --skip-torch-cuda-test --listen --xformers --enable-insecure-extension-access --theme dark --clip-models-path /content/sdwebui/models/CLIP --gradio-queue --multiple

Python 3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]
Commit hash: 22bcc7be428c94e9408f589966c2040187245d81
Installing requirements for Web UI






Launching Web UI with arguments: --skip-torch-cuda-test --listen --xformers --enable-insecure-extension-access --theme dark --clip-models-path /content/sdwebui/models/CLIP --gradio-queue --multiple
/usr/local/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
[AddNet] Updating model hashes...
100% 1/1 [00:00<00:00,  3.33it/s]
[AddNet] Updating model hashes...
100% 1/1 [00:00<00:00, 1400.90it/s]
all detected, remote.moe trying to connect...
all detected, cloudflared trying to connect...
Download cloudflared...: 100% 34.5M/34.5M [00:00<00:00

# Misc
Kumpulan tambahan untuk download Checkpoint, Lora, Textual Inversion ke dalam Gdrive.

In [ ]:
#@title Download Checkpoint { display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2

#@markdown Isi dengan link download
model_url = "https://civitai.com/api/download/models/29460" #@param {type:"string"}

#@markdown Isi dengan nama yang di inginkan
model_name = "RealisticVisionV2.0.safetensors" #@param {type:"string"}

model_url = f"\"{model_url}\""
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d /content/drive/MyDrive/SDdata/Checkpoint -o {model_name}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
889085|OK  |   125MiB/s|/content/drive/MyDrive/SDdata/Checkpoint/RealisticVisionV2.0.safetensors

Status Legend:
(OK):download completed.


In [ ]:
#@title Download Lora { display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2

#@markdown Isi dengan link download
model_url = "https://huggingface.co/aimainia/japaneseDollLikeness_v10/resolve/main/japaneseDollLikeness_v10.safetensors" #@param {type:"string"}

#@markdown Isi dengan nama yang di inginkan
model_name = "japaneseDollLikeness_v10.safetensors" #@param {type:"string"}

model_url = f"\"{model_url}\""
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d /content/drive/MyDrive/SDdata/Lora -o {model_name}

In [ ]:
#@title Download Textual Inversion { display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2

#@markdown Isi dengan link download
model_url = "https://civitai.com/api/download/models/5119" #@param {type:"string"}

#@markdown Isi dengan nama yang di inginkan
model_name = "pureerosface_v1.pt" #@param {type:"string"}

model_url = f"\"{model_url}\""
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d /content/drive/MyDrive/SDdata/Embeddings -o {model_name}